In [ ]:
# Google Colab Setup: Ensure necessary libraries are installed
!pip install scikit-learn pandas matplotlib

# Import necessary libraries
from google.colab import files
import os
import zipfile
import pandas as pd
import re
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score
import matplotlib.pyplot as plt

print("Libraries imported successfully!")

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Step 1: Upload and Extract the ZIP File
uploaded_zip_path = '/content/drive/MyDrive/CIS_694/Combined_Spam_Ham.zip'  # Path for the uploaded ZIP file
extracted_folder_path = '/content/drive/MyDrive/CIS_694/'

# Unzip the file
print("Unzipping the file...")
with zipfile.ZipFile(uploaded_zip_path, 'r') as zip_ref:
    zip_ref.extractall(extracted_folder_path)

print(f"Files extracted to: {extracted_folder_path}")

In [ ]:
# Step 2: Load Data and Extract Features
email_folder_path = os.path.join(extracted_folder_path, '/content/drive/MyDrive/CIS_694/Combined_Spam_Ham')

email_data = []
print("Loading and extracting features from email files...")
for email_file in os.listdir(email_folder_path):
    with open(os.path.join(email_folder_path, email_file), 'r', encoding='utf-8', errors='ignore') as file:
        content = file.read()
        sender = re.search(r"From: (.*)", content)
        receiver = re.search(r"To: (.*)", content)
        subject = re.search(r"Subject: (.*)", content)
        body_start = content.find("\n\n")
        body = content[body_start:].strip() if body_start != -1 else ''
        email_data.append({
            'FileName': email_file,
            'Sender': sender.group(1).strip() if sender else None,
            'Receiver': receiver.group(1).strip() if receiver else None,
            'Subject': subject.group(1).strip() if subject else None,
            'Body': body.strip()
        })

email_df = pd.DataFrame(email_data)

# Print DataFrame size and head/tail
print(f"Original DataFrame: {email_df.shape[0]} rows, {email_df.shape[1]} columns")
print("DataFrame head:")
print(email_df.head())
print("DataFrame tail:")
print(email_df.tail())

In [ ]:
# Save the DataFrame to a CSV file
output_csv_path = os.path.join(extracted_folder_path, '/content/drive/MyDrive/CIS_694/email_features.csv')
email_df.to_csv(output_csv_path, index=False, encoding='utf-8')

# Confirm the file has been saved
print(f"CSV file saved at: {output_csv_path}")

In [ ]:
# Download the CSV file
files.download(output_csv_path)

In [ ]:
print("Data types of features:")
print(email_df.dtypes)

In [ ]:
import re
from nltk.corpus import stopwords
import nltk

# Download stopwords from NLTK (if not already done)
nltk.download('stopwords')

def clean_text(text):
    # Step 1: Remove HTML tags
    text = re.sub(r'<[^>]+>', ' ', text)  # Removes HTML tags like <html>, <body>, etc.
    # Step 2: Remove non-alphanumeric characters (except spaces)
    text = re.sub(r'[^\w\s]', ' ', text)
    # Step 3: Remove extra spaces
    text = re.sub(r'\s+', ' ', text)
    # Step 4: Remove email addresses
    text = re.sub(r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}\b', '', text)
    # Step 5: Remove URLs
    text = re.sub(r'http\S+|www\S+|https\S+', '', text)
    # Step 6: Remove numbers
    text = re.sub(r'\d+', '', text)
    # Step 7: Remove programming keywords or system-specific tokens
    text = re.sub(r'\b(html|body|font|color|size|center|pythonnet|cbgovernmentgw|gward)\b', '', text, flags=re.IGNORECASE)
    # Step 8: Convert to lowercase
    text = text.lower()
    # Step 9: Remove stopwords
    stop_words = set(stopwords.words('english'))
    words = text.split()
    filtered_words = [word for word in words if word not in stop_words]
    # Step 10: Join the cleaned words back into a single string
    cleaned_text = ' '.join(filtered_words)
    return cleaned_text.strip()

# Apply the updated clean_text function to the 'Body' column
email_df['Cleaned_Body'] = email_df['Body'].apply(clean_text)

# Create a new DataFrame with preprocessed columns
preprocessed_email_df = email_df[['FileName', 'Sender', 'Receiver', 'Subject', 'Cleaned_Body']].copy()

# Print preprocessed DataFrame size and head/tail for verification
print(f"Preprocessed DataFrame: {preprocessed_email_df.shape[0]} rows, {preprocessed_email_df.shape[1]} columns")
print("Preprocessed DataFrame head:")
print(preprocessed_email_df.head())
print("Preprocessed DataFrame tail:")
print(preprocessed_email_df.tail())


In [ ]:
preprocessed_email_df.to_csv('/content/drive/MyDrive/CIS_694/Preprocessed_emails.csv', index=False)
print("Preprocessed DataFrame saved as 'Preprocessed_emails.csv'")
files.download('/content/drive/MyDrive/CIS_694/Preprocessed_emails.csv')

In [ ]:
# Step 4: Bootstrapping to Create a Larger Dataset
desired_rows = 15000
bootstrapped_df = preprocessed_email_df.sample(n=desired_rows, replace=True, random_state=42).reset_index(drop=True)

print(f"Bootstrapped DataFrame: {bootstrapped_df.shape[0]} rows, {bootstrapped_df.shape[1]} columns")
print("Bootstrapped DataFrame head:")
print(bootstrapped_df.head())
print("Bootstrapped DataFrame tail:")
print(bootstrapped_df.tail())

In [ ]:
print(f"Bootstrapped DataFrame: {bootstrapped_df.shape[0]} rows, {preprocessed_email_df.shape[1]} columns")


In [ ]:
# Handle missing values by removing rows with null values
print("Handling missing values by removing rows with null values...")

# Check for any null values in required columns
print("Before dropping null values:")
print(bootstrapped_df.isnull().sum())

# Drop rows where any of the key columns ('Sender', 'Subject', 'Cleaned_Body') are null
bootstrapped_df = bootstrapped_df.dropna(subset=['Sender', 'Subject', 'Cleaned_Body']).reset_index(drop=True)

# Recreate the 'Combined_Text' column after dropping null rows
bootstrapped_df['Combined_Text'] = (
    bootstrapped_df['Sender'] + " " +
    bootstrapped_df['Subject'] + " " +
    bootstrapped_df['Cleaned_Body']
)

# Ensure all entries in 'Combined_Text' are strings
bootstrapped_df['Combined_Text'] = bootstrapped_df['Combined_Text'].astype(str)

# Check if all values are strings and no NaN values exist
print("After dropping null values:")
print(bootstrapped_df.isnull().sum())
print("Sample Combined_Text column data:")
print(bootstrapped_df['Combined_Text'].head())
print("Are all entries strings?", bootstrapped_df['Combined_Text'].apply(lambda x: isinstance(x, str)).all())

In [ ]:
print(f"DataFrame size after bootstrapping and handling null values: {bootstrapped_df.shape[0]} rows, {bootstrapped_df.shape[1]} columns")

In [ ]:
# Step 5: Add Class Labels Based on Extended Spam Keywords
def classify_email(subject, body):
    spam_keywords = [
        'win', 'free', 'urgent', 'offer', 'loan', 'click', 'subscribe',
        'money', 'cash', 'credit', 'gift', 'winner', 'deal', 'buy now',
        'limited time', 'discount', 'promotion', 'guarantee', 'trial', 'stop', 'end','exclusive', 'claim now'
    ]
    content = f"{subject} {body}".lower()
    return int(any(keyword in content for keyword in spam_keywords))

bootstrapped_df['Spam'] = bootstrapped_df.apply(
    lambda row: classify_email(row['Subject'], row['Cleaned_Body']), axis=1
)

# Print updated DataFrame head/tail
print("Bootstrapped DataFrame with Spam labels head:")
print(bootstrapped_df.head())
print("Bootstrapped DataFrame with Spam labels tail:")
print(bootstrapped_df.tail())

In [ ]:
bootstrapped_df.to_csv('/content/drive/MyDrive/CIS_694/Class_label_emails.csv', index=False)
print("Bootstrapped DataFrame with class labels saved as 'Class_label_emails.csv'")
files.download('/content/drive/MyDrive/CIS_694/Class_label_emails.csv')

In [ ]:
 #!pip install --upgrade --force-reinstall numpy==1.26.4 scipy==1.13.1 gensim

In [ ]:
!pip install gensim
from gensim.models import Word2Vec
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Step 6: Prepare Data for Model Training using Word2Vec
tokenized_texts = bootstrapped_df['Combined_Text'].apply(lambda x: x.split())

# Train a Word2Vec model on the tokenized texts
word2vec_model = Word2Vec(sentences=tokenized_texts, vector_size=100, window=5, min_count=1, workers=4)

# Function to vectorize text using Word2Vec
def vectorize_texts(texts, model):
    vectors = []
    for text in texts:
        tokens = text.split()
        token_vectors = [model.wv[word] for word in tokens if word in model.wv]
        if token_vectors:
            vectors.append(np.mean(token_vectors, axis=0))
        else:
            vectors.append(np.zeros(model.vector_size))
    return np.array(vectors)

# Vectorize the data
X_bootstrapped = vectorize_texts(bootstrapped_df['Combined_Text'], word2vec_model)
y_bootstrapped = bootstrapped_df['Spam']


In [ ]:
# Step 7: Split Data for Training and Testing
X_train, X_test, y_train, y_test = train_test_split(
    X_bootstrapped, y_bootstrapped, test_size=0.3, random_state=42)

In [ ]:
# Step 8: Train and Evaluate Multiple Models
models = {
    "Random Forest": RandomForestClassifier(random_state=42, n_estimators=100),
    "Logistic Regression": LogisticRegression(random_state=42, max_iter=1000),
    "SVM": SVC(random_state=42)
}

results = []

for model_name, model in models.items():
    print(f"Training {model_name}...")
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    acc = accuracy_score(y_test, y_pred)
    report = classification_report(y_test, y_pred, output_dict=True)
    results.append({
        'Model': model_name,
        'Accuracy': acc,
        'Precision': report['weighted avg']['precision'],
        'Recall': report['weighted avg']['recall'],
        'F1-Score': report['weighted avg']['f1-score']
    })

# Convert results to DataFrame
results_df = pd.DataFrame(results)

# Print and visualize the results
print("Model Performance Comparison:")
print(results_df)

plt.figure(figsize=(10, 6))
for metric in ['Accuracy', 'Precision', 'Recall', 'F1-Score']:
    plt.plot(results_df['Model'], results_df[metric], marker='o', label=metric)
plt.title('Model Performance Metrics')
plt.xlabel('Model')
plt.ylabel('Score')
plt.legend()
plt.grid()
plt.show()

In [ ]:
# Step 9: Use the Best Model for Future Predictions

best_model_name = results_df.sort_values(by='Accuracy', ascending=False).iloc[0]['Model']
print(f"The best performing model is: {best_model_name}")
best_model = models[best_model_name]


# Example Prediction
#example_email = "Ok it's Thanksgiving - so you deserve a FEAST today.And like at Thanksgiving dinner, instead of one main course (like our usual weekly essays), today you're getting a full spread:10 quick-hitter ideas that will make your career better for having read them. A mix of unconventional career strategies, job search hacks, and things I wish I knew earlier.I think you'll like the format — and you'll probably save a thing or three that you'll use forever.Before we dig in…I kinda want to start with the dessert first:"
#example_email = "Some Grading Errors Have Been Found in Short Exam 2. They all have been corrected now. Some of scores of the Short Exam2 have been updated (mostly lowered) now"
#example_vectorized = vectorizer.transform([example_email])
#prediction = best_model.predict(example_vectorized)
#print(f"The example email is classified as: {'Ham' if prediction[0] == 1 else 'Spam'}")

In [ ]:
from ipywidgets import Text, Button, Output, VBox
from IPython.display import display
import numpy as np

# Function to vectorize a single input text using Word2Vec
def vectorize_single_text(text, model):
    tokens = text.split()
    token_vectors = [model.wv[word] for word in tokens if word in model.wv]
    if token_vectors:
        return np.mean(token_vectors, axis=0)
    else:
        return np.zeros(model.vector_size)

# Create widgets for user interaction
text_input = Text(description="Email:", placeholder="Enter email content here...")
predict_button = Button(description="Classify")
output = Output()

# Define prediction function
def predict_email_type(change):
    email_content = text_input.value
    if not email_content.strip():
        with output:
            output.clear_output()
            print("Please enter a valid email.")
        return

    # Vectorize the input email
    email_vectorized = vectorize_single_text(email_content, word2vec_model).reshape(1, -1)

    # Predict using the trained model
    prediction = best_model.predict(email_vectorized)
    with output:
        output.clear_output()
        print(f"The email is classified as: {'Spam' if prediction[0] == 1 else 'Ham'}")

# Attach the handler to the button
predict_button.on_click(predict_email_type)

# Display the UI
ui = VBox([text_input, predict_button, output])
display(ui)
